In [1]:
!mamba install pandas
!mamba install seaborn 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
from datetime import datetime
import os
from pathlib import Path

mambajs 0.19.13

Specs: xeus-python, numpy, matplotlib, pillow, ipywidgets>=8.1.6, ipyleaflet, scipy, pandas
Channels: emscripten-forge, conda-forge

Solving environment...
Solving took 0.747 seconds
  Name                          Version                       Build                         Channel                       
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
+ pandas                        3.0.1                         np22py313h9d9dc1e_0           emscripten-forge              
+ python-tzdata                 2025.3                        pyhd8ed1ab_0                  conda-forge                   
- pip                           25.3                          pyh145f28c_0                  conda-forge                   
mambajs 0.19.13

Specs: xeus-python, numpy, matplotlib, pillow, ipywidgets>=8.1.6, ipyleaflet, scipy, pandas, seaborn
Channels: emscripten-forge, conda-forge

Solving environment...
Solvi

In [11]:
def load_titanic_data(): 
    url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
    try:
        df = pd.read_csv(url)
        print(f"Загружено {len(df)} записей через GitHub")
        return df
    except Exception as e2:
        print(f"Ошибка загрузки через GitHub: {e2}")
        print("Создаем демонстрационные данные...")
    return create_sample_titanic_data()

def create_sample_titanic_data():
    """
    Создание демонстрационных данных о Титанике
    """
    np.random.seed(42)
    n_samples = 891
    
    # Создаем основные колонки
    data = {
        'passenger_id': range(1, n_samples + 1),
        'survived': np.random.choice([0, 1], n_samples, p=[0.62, 0.38]),
        'pclass': np.random.choice([1, 2, 3], n_samples, p=[0.24, 0.21, 0.55]),
        'sex': np.random.choice(['male', 'female'], n_samples, p=[0.65, 0.35]),
        'age': np.random.normal(30, 14, n_samples).clip(0.5, 80).astype(int),
        'sibsp': np.random.choice([0, 1, 2, 3, 4, 5], n_samples, p=[0.68, 0.23, 0.06, 0.02, 0.01, 0.005]),
        'parch': np.random.choice([0, 1, 2, 3, 4, 5], n_samples, p=[0.76, 0.13, 0.09, 0.01, 0.005, 0.005]),
        'fare': np.random.exponential(32, n_samples).clip(0, 512).round(2),
        'embarked': np.random.choice(['C', 'Q', 'S'], n_samples, p=[0.19, 0.09, 0.72])
    }
    
    df = pd.DataFrame(data)
    
    # Добавляем логические зависимости
    # Женщины выживают чаще
    df.loc[df['sex'] == 'female', 'survived'] = np.random.choice([0, 1], sum(df['sex'] == 'female'), p=[0.25, 0.75])
    # 1-й класс выживает чаще
    df.loc[df['pclass'] == 1, 'survived'] = np.random.choice([0, 1], sum(df['pclass'] == 1), p=[0.35, 0.65])
    # Дети выживают чаще
    df.loc[df['age'] < 16, 'survived'] = np.random.choice([0, 1], sum(df['age'] < 16), p=[0.4, 0.6])
    
    return df
df_titanic = load_titanic_data()

Загружено 891 записей через GitHub


In [12]:
print(f"\nРазмер данных: {df_titanic.shape[0]} строк, {df_titanic.shape[1]} столбцов")

print(f"\nТипы данных:")
print(df_titanic.dtypes)

print(f"\nОсновные статистики:")
print(df_titanic.describe(include='all'))

print(f"\nПропущенные значения:")
missing_data = df_titanic.isnull().sum()
print(missing_data[missing_data > 0])

print(f"\nПервые 5 записей:")
print(df_titanic.head())



Размер данных: 891 строк, 12 столбцов

Типы данных:
PassengerId      int64
Survived         int64
Pclass           int64
Name               str
Sex                str
Age            float64
SibSp            int64
Parch            int64
Ticket             str
Fare           float64
Cabin              str
Embarked           str
dtype: object

Основные статистики:
        PassengerId    Survived      Pclass                     Name   Sex  \
count    891.000000  891.000000  891.000000                      891   891   
unique          NaN         NaN         NaN                      891     2   
top             NaN         NaN         NaN  Braund, Mr. Owen Harris  male   
freq            NaN         NaN         NaN                        1   577   
mean     446.000000    0.383838    2.308642                      NaN   NaN   
std      257.353842    0.486592    0.836071                      NaN   NaN   
min        1.000000    0.000000    1.000000                      NaN   NaN   
25%      22

In [15]:
print("СОЗДАНИЕ SQLite БАЗЫ ДАННЫХ")
print("="*60)

db_dir = Path("database")
db_dir.mkdir(exist_ok=True)
db_path = db_dir / "titanic.db"

if db_path.exists():
    os.remove(db_path)
    print(f"Удалена существующая база данных")

conn = sqlite3.connect(db_path)
cursor = conn.cursor()
print(f"Создана база данных: {db_path}")

df_titanic.to_sql('passengers', conn, if_exists='replace', index=False)
print(f"Данные загружены в таблицу 'passengers'")
print(f"Количество записей в таблице: {pd.read_sql('SELECT COUNT(*) FROM passengers', conn).iloc[0, 0]}")

СОЗДАНИЕ SQLite БАЗЫ ДАННЫХ
Удалена существующая база данных
Создана база данных: database/titanic.db


<class 'pandas.errors.DatabaseError'>: Execution failed

In [16]:
print("\n Общая статистика выживаемости:")
query = """
SELECT 
    COUNT(*) as total_passengers,
    SUM(survived) as survived,
    ROUND(AVG(survived) * 100, 2) as survival_rate_percent
FROM passengers
"""
df_result = pd.read_sql(query, conn)
print(df_result)

print("\n Выживаемость по классу обслуживания:")
query = """
SELECT 
    pclass,
    COUNT(*) as total,
    SUM(survived) as survived,
    ROUND(AVG(survived) * 100, 2) as survival_rate_percent
FROM passengers
GROUP BY pclass
ORDER BY pclass
"""
df_pclass = pd.read_sql(query, conn)
print(df_pclass)

print("\n Выживаемость по полу:")
query = """
SELECT 
    sex,
    COUNT(*) as total,
    SUM(survived) as survived,
    ROUND(AVG(survived) * 100, 2) as survival_rate_percent
FROM passengers
GROUP BY sex
"""
df_sex = pd.read_sql(query, conn)
print(df_sex)

print("\n Выживаемость по возрастным группам:")
query = """
SELECT 
    CASE 
        WHEN age < 18 THEN 'Дети (0-17)'
        WHEN age BETWEEN 18 AND 30 THEN 'Молодежь (18-30)'
        WHEN age BETWEEN 31 AND 50 THEN 'Средний возраст (31-50)'
        WHEN age > 50 THEN 'Пожилые (50+)'
        ELSE 'Неизвестно'
    END as age_group,
    COUNT(*) as total,
    SUM(survived) as survived,
    ROUND(AVG(survived) * 100, 2) as survival_rate_percent
FROM passengers
WHERE age IS NOT NULL
GROUP BY age_group
ORDER BY 
    CASE age_group
        WHEN 'Дети (0-17)' THEN 1
        WHEN 'Молодежь (18-30)' THEN 2
        WHEN 'Средний возраст (31-50)' THEN 3
        WHEN 'Пожилые (50+)' THEN 4
        ELSE 5
    END
"""
df_age = pd.read_sql(query, conn)
print(df_age)

print("\n Выживаемость по полу и классу:")
query = """
SELECT 
    sex,
    pclass,
    COUNT(*) as total,
    SUM(survived) as survived,
    ROUND(AVG(survived) * 100, 2) as survival_rate_percent
FROM passengers
GROUP BY sex, pclass
ORDER BY sex, pclass
"""
df_sex_class = pd.read_sql(query, conn)
print(df_sex_class)

print("\n Выживаемость по порту посадки:")
query = """
SELECT 
    embarked,
    COUNT(*) as total,
    SUM(survived) as survived,
    ROUND(AVG(survived) * 100, 2) as survival_rate_percent
FROM passengers
WHERE embarked IS NOT NULL
GROUP BY embarked
ORDER BY survival_rate_percent DESC
"""
df_embarked = pd.read_sql(query, conn)
print(df_embarked)


 Общая статистика выживаемости:


<class 'pandas.errors.DatabaseError'>: Execution failed on sql '
SELECT 
    COUNT(*) as total_passengers,
    SUM(survived) as survived,
    ROUND(AVG(survived) * 100, 2) as survival_rate_percent
FROM passengers
': database disk image is malformed